# TODO:
- [ ] refactor frame creation
- [ ] save labels and label map in vid folder
- [ ] alert if vid with same name already exists
- [ ]  fix logging

# Setup

## imports

In [1]:
import os
import sys
import time
import json
from shutil import copy
from sklearn.utils import shuffle
import datetime as dt

In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D, MaxPooling2D)
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [3]:
import cv2

In [4]:
from contextlib import redirect_stdout

In [5]:
# setup matplotlib to display plots in the notebook
# %matplotlib inline

# third party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setup display options
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 400
pd.options.display.float_format = '{:,.5g}'.format
np.set_printoptions(precision=5, suppress=False)

# setup seaborn to use matplotlib defaults & styles
sns.set()
sns.set(font_scale=1.2)
sns.set_style("whitegrid", {'axes.grid' : False})

## paths

In [6]:
pwd = os.path.dirname(os.getcwd()) + '/'
pwd

'/mnt/seals/'

In [7]:
path_cache = pwd + 'cache/'
path_models = pwd + 'models/'

In [8]:
# for constructing vids in cache REFACTOR
path_data = pwd + 'data/images/'

In [9]:
# folder where we'll store each vid grouped into folders
path_vids = path_cache + 'vids/'

## setup logging

In [10]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])

logger = logging.getLogger()

# helper functions

In [11]:
def plot_pic(imgpath):
    plt.imshow(plt.imread(imgpath))
    plt.show()

# Convert data back into sequence for video classification

## read list of paths across train/val/test and seal / no seal folders

In [12]:
path_data

'/mnt/seals/data/images/'

In [13]:
paths_jpgs = []
for folder, subs, files in os.walk(path_data):        
    for filename in files:
        if filename[-4:] == '.jpg' or  filename[-4:] == 'jpeg':
            paths_jpgs.append(os.path.abspath(os.path.join(folder, filename)))

In [14]:
# create dataframe from paths
dfp = pd.DataFrame(paths_jpgs)
dfp.columns = ['path']
dfp['label'] = dfp['path'].str.split("/").str.get(-2)
dfp['filename'] = dfp['path'].str.split("/").str.get(-1)
dfp.sort_values("filename", inplace=True)
dfp.reset_index(inplace=True,drop=True)
dfp['vid'] = dfp['filename'].str.split("-").str.get(0) + '-' + dfp['filename'].str.split("-").str.get(1)
dfp['seal'] = pd.get_dummies(dfp['label'])['seal']
dfp.to_csv(path_vids + "df.csv")

In [15]:
vids = list(dfp['vid'].unique())

# Create train/test split

## functions to load precomputed sequence data for list of vids

In [16]:
def get_sequence_data_for_vids(list_of_vid_names, sequence_length, pretrained_model_name, pooling):
    """
    Load precomputed sequence features data of given length together with targets [returns data later to train/eval models: x, y]
    for list of vid names and concatenate into one long array
    
    Args:
        list_of_vid_names: name of vid (should already have frames in folder in `/cache/vids/*vid_name*/frames/`)
        sequence_length: length of sequence to fetch precomputed features for
        pretrained_model_name: name of pretrained model whose features should be loaded (assuming these were already precomputed)
        pooling: pooling method used with pretrained model
    
    Returns:
        sequence_features_array_for_all_vids, sequence_targets_array_for_all_vids
    
    """
    
    # create clips of length NUM_FRAMES
    x = []
    y = []
    
    for v, vid_name in enumerate(list_of_vid_names):
        
        path_sequences_features = path_vids + vid_name + '/sequences/features_sequence_' + str(sequence_length) + '_' + pretrained_model_name + '_' + pooling + 'pooling.npy'
        path_sequences_targets = path_vids + vid_name + '/sequences/targets_sequence_' + str(sequence_length) + '.npy'

        # load precomputed features
        features = np.load(path_sequences_features)
        targets = np.load(path_sequences_targets)
        
        x.extend(features)
        y.extend(targets)

    return np.array(x), np.array(y)

## generate train / test split

> todo: write functions to make this quicker

> todo: cross-validation

> todo: experiments with config files and json results with function to aggregate to dataframe

In [17]:
np.random.seed(1337)

In [18]:
ids = list(range(0,len(vids)))
np.random.shuffle(ids)

In [19]:
str(ids)

'[16, 0, 13, 41, 5, 12, 44, 29, 21, 33, 31, 10, 7, 37, 32, 4, 3, 15, 17, 14, 19, 30, 11, 36, 35, 2, 38, 22, 34, 27, 1, 24, 45, 43, 6, 9, 8, 20, 18, 26, 42, 25, 39, 40, 28, 23]'

In [20]:
vids_train = [vids[c] for c in ids[0:40]]
vids_valid = [vids[c] for c in ids[40:44]]
vids_test = [vids[c] for c in ids[44:]]

In [21]:
vids_valid

['s6-1247', 's32-3110', 's45-6301', 's46-8087']

In [22]:
vids_test

['s35-3664', 's30-516']

# Fit models

In [42]:
def fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling):

    ###########################
    ### create folder for model 
    ###########################

    path_model = path_models + str(model_id) + '/'
    if not os.path.exists(path_model):
        os.makedirs(path_model)

    ###########################
    ### create train/test split
    ###########################

    x_train, y_train = get_sequence_data_for_vids(vids_train, sequence_length, pretrained_model_name, pooling)
    x_valid, y_valid = get_sequence_data_for_vids(vids_valid, sequence_length, pretrained_model_name, pooling)
    x_test, y_test = get_sequence_data_for_vids(vids_test, sequence_length, pretrained_model_name, pooling)

    # shuffle test and train batches
    x_train, y_train = shuffle(x_train, y_train)
    x_valid, y_valid = shuffle(x_valid, y_valid)

    # CREATE CLASS BALANCE
#     y_train = pd.DataFrame(y_train)
#     keeps = list(y_train[y_train[0]==1].head(int(len(y_train[y_train[0]==1].index)/2)).index)
#     keeps2 = list(y_train[y_train[0] == 0].index)
#     keeps.extend(keeps2)
#     #
#     y_train = y_train.iloc[keeps]
#     y_train = y_train.values
#     x_train = x_train[keeps,:,:]

    NUM_CLASSES = y_train.shape[1]
    NUM_FEATURES = x_train.shape[2]
    SEQ_LENGTH = x_train.shape[1]
    
#     # reshape sequence length 1 into image features if not a sequence
#     if sequence_length == 1:
#         x_train = np.squeeze(x_train, axis=1)
#         x_valid = np.squeeze(x_valid, axis=1)
#         x_test = np.squeeze(x_test, axis=1)



    ##############################
    ### keep track of model params
    ##############################

    # create dict with model parameters
    results = {}

    results['id'] = str(model_id)

    NUM_EPOCHS = 50
    BATCH_SIZE = 32
    results['fit_batch_size'] = BATCH_SIZE
    
    results['fit_num_classes'] = NUM_CLASSES
    results['model_num_features'] = NUM_FEATURES
    results['model_sequence_length'] = SEQ_LENGTH
    
    results['pretrained_model_name'] = pretrained_model_name
    results['pretrained_model_pooling'] = pooling

    results['shape_y_train'] = str(y_train.shape)
    results['shape_x_train'] = str(x_train.shape)
    results['shape_x_valid'] = str(x_valid.shape)
    results['shape_x_test'] = str(x_test.shape)

    results['model_architecture'] = architecture
    results['model_layer_1_sizefactor'] = layer_1_sizefactor
    results['model_layer_2_sizefactor'] = layer_2_sizefactor
    results['model_layer_3_sizefactor'] = layer_3_sizefactor
    results['model_dropout'] = dropout

    ################
    ### define model
    ################

    if architecture == "LSTM":
        # https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
        model = Sequential()

        # layer 1 (LSTM layer)
        model.add(LSTM(NUM_FEATURES//layer_1_sizefactor, return_sequences=False, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))

        # layer 2 (dense)
        if layer_2_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))

        # layer 3 (dense)
        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    if architecture == 'MLP':
        model = Sequential()
        model.add(Flatten(input_shape=(SEQ_LENGTH, NUM_FEATURES)))
        model.add(Dense(NUM_FEATURES//2, activation='relu'))

        if layer_2_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    # save model summary to file
    with open(path_model + 'model_summary.txt', 'w') as f:
        with redirect_stdout(f):
            model.summary()

    # track number of params in model
    results['param_count'] = model.count_params()

    #############
    ### fit model
    #############

    # setup training callbacks
    stopper_patience = 10
    results['fit_stopper_patience'] = stopper_patience
    callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
    callback_csvlogger = CSVLogger(path_model + 'training.log')
    callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                                 save_best_only=True, verbose=0)
    
    # start time training
    start = dt.datetime.now()
    results['fit_train_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    history = model.fit(x_train, y_train, 
              validation_data=(x_valid,y_valid),
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
              shuffle=True,
              verbose=0)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_train_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_train_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # get number of epochs actually trained (might have early stopped)
    epochs_trained = 0
    epochs_trained = callback_stopper.stopped_epoch - stopper_patience

    results['fit_stopped_early'] = True
    if epochs_trained == 0 and len(history.history) > stopper_patience:
        results['fit_stopped_early'] = False
        epochs_trained = NUM_EPOCHS - 1 

    results['fit_num_epochs'] = epochs_trained
    results['fit_val_acc'] = history.history['val_acc'][epochs_trained]
    results['fit_train_acc'] = history.history['acc'][epochs_trained]
    results['fit_val_loss'] = history.history['val_loss'][epochs_trained]
    results['fit_train_loss'] = history.history['loss'][epochs_trained]
    

    #######################
    ### predict on test set
    #######################
    
    
    # start time inference
    start = dt.datetime.now()
    results['fit_test_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    # calculate predictions on test set
    predictions = model.predict(x_test)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_test_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_test_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # calculate test error 
    pdf = pd.DataFrame(predictions)
    pdf.columns = ['noseal','seal']

    # get filenames for predictions
    filenames = []
    for vid_name in vids_test:
        filenames_vid = list(dfp[dfp['vid'] == vid_name]['path'])
        filenames.extend(filenames_vid[sequence_length-1:])
    pdf['filename'] = filenames
    truth = pd.DataFrame(y_test)
    truth.columns = ['truth_noseal','truth_seal']
    truth = truth[['truth_seal']]
    pdf['prediction'] = pdf['seal'].apply(lambda x: round(x))
    pdf = pd.concat([pdf, truth], axis=1)
    pdf['error'] = (pdf['prediction'] != pdf['truth_seal']).astype(int)

    test_acc = 1 - pdf['error'].mean()

    pdf.to_csv(path_model + 'test_predictions.csv')

    results['fit_test_acc'] = 1 - pdf['error'].mean()
    logger.info("model {} test acc: {}".format(model_id, test_acc))

    ###################################
    ### save experiment results to file
    ###################################
    # log results
    logger.info(json.dumps(results))
    
    with open(path_model + 'params.json', 'w') as f:
        json.dump(results, f)

In [43]:
# pretrained_model_names = ["inception_resnet_v2", "inception_v3", "mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
# poolings = ['avg','max']
# sequence_lengths = [1, 3, 5, 10, 15, 20, 40]

# architectures = ['LSTM', "MLP"]
# layer_1_sizefactors = [1,2,4,8]
# layer_2_sizefactors = [0,1,2,4,8]
# layer_3_sizefactors = [0,1,2,4,8]
# dropouts = [0, 0.1, 0.2,0.3,0.4,0.5]

In [44]:
pretrained_model_names = ["inception_resnet_v2"]
poolings = ['avg']
sequence_lengths = [1, 3]

architectures = ["MLP", "LSTM"]
layer_1_sizefactors = [1,2,4,8]
layer_2_sizefactors = [0,2,4,8]
layer_3_sizefactors = [0,2,4,8]
dropouts = [0.2, 0.5]

In [45]:
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                experiment_count_total+=1
experiment_count_total

512

In [ ]:
model_id = 1
experiment_count = 1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                
                                experiment_count += 1
                                model_id+=1

2018-12-17 11:42:49,111 [MainThread  ] [INFO ]  begin experiment 341/512 - model_id: 341, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 4, layer_3_sizefactor: 4, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 11:45:51,103 [MainThread  ] [INFO ]  model 341 test acc: 0.8461538461538461
2018-12-17 11:45:51,105 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9308996088657105, "fit_test_dt_duration": "1", "fit_test_acc": 0.8461538461538461, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "179", "fit_val_loss": 0.24571322989380914, "fit_train_dt_end": "2018-12-17 11:45:49", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 11:45:49", "model_dropout": 0.2, "model_layer_1_sizefactor": 4, "fit_train_dt_start": "2018-12-17 11:42:50", "fit_batch_size": 32, "fit_train_acc": 0.9023141511365964, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 4

2018-12-17 11:59:24,392 [MainThread  ] [INFO ]  begin experiment 347/512 - model_id: 347, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 2, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 12:01:23,426 [MainThread  ] [INFO ]  model 347 test acc: 0.939799331103679
2018-12-17 12:01:23,427 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9252498914033939, "fit_test_dt_duration": "1", "fit_test_acc": 0.939799331103679, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "116", "fit_val_loss": 0.32739054184020056, "fit_train_dt_end": "2018-12-17 12:01:21", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 12:01:21", "model_dropout": 0.2, "model_layer_1_sizefactor": 4, "fit_train_dt_start": "2018-12-17 11:59:25", "fit_batch_size": 32, "fit_train_acc": 0.868011468359615, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 2, "

2018-12-17 12:15:14,587 [MainThread  ] [INFO ]  begin experiment 353/512 - model_id: 353, architecture: LSTM, layer_1_sizefactor: 8, layer_2_sizefactor: 0, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 12:17:07,762 [MainThread  ] [INFO ]  model 353 test acc: 0.9096989966555185
2018-12-17 12:17:07,764 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9282920469879223, "fit_test_dt_duration": "2", "fit_test_acc": 0.9096989966555185, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "110", "fit_val_loss": 0.2667575466119948, "fit_train_dt_end": "2018-12-17 12:17:05", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 12:17:05", "model_dropout": 0.2, "model_layer_1_sizefactor": 8, "fit_train_dt_start": "2018-12-17 12:15:15", "fit_batch_size": 32, "fit_train_acc": 0.8894122465819383, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 0,

2018-12-17 12:28:20,475 [MainThread  ] [INFO ]  begin experiment 359/512 - model_id: 359, architecture: LSTM, layer_1_sizefactor: 8, layer_2_sizefactor: 0, layer_3_sizefactor: 8, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 12:30:13,696 [MainThread  ] [INFO ]  model 359 test acc: 0.862876254180602
2018-12-17 12:30:13,697 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9282920467029805, "fit_test_dt_duration": "2", "fit_test_acc": 0.862876254180602, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "109", "fit_val_loss": 0.23127041289543182, "fit_train_dt_end": "2018-12-17 12:30:10", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 12:30:10", "model_dropout": 0.2, "model_layer_1_sizefactor": 8, "fit_train_dt_start": "2018-12-17 12:28:21", "fit_batch_size": 32, "fit_train_acc": 0.8838828588982183, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 8, 

2018-12-17 12:42:26,346 [MainThread  ] [INFO ]  begin experiment 365/512 - model_id: 365, architecture: LSTM, layer_1_sizefactor: 8, layer_2_sizefactor: 2, layer_3_sizefactor: 4, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 12:46:16,560 [MainThread  ] [INFO ]  model 365 test acc: 0.8695652173913043
2018-12-17 12:46:16,561 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9378531071114986, "fit_test_dt_duration": "3", "fit_test_acc": 0.8695652173913043, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "226", "fit_val_loss": 0.20424605651181038, "fit_train_dt_end": "2018-12-17 12:46:13", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 12:46:13", "model_dropout": 0.2, "model_layer_1_sizefactor": 8, "fit_train_dt_start": "2018-12-17 12:42:27", "fit_batch_size": 32, "fit_train_acc": 0.9047716567683801, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 4

2018-12-17 13:01:15,962 [MainThread  ] [INFO ]  begin experiment 371/512 - model_id: 371, architecture: LSTM, layer_1_sizefactor: 8, layer_2_sizefactor: 4, layer_3_sizefactor: 2, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 13:04:59,837 [MainThread  ] [INFO ]  model 371 test acc: 0.8193979933110368
2018-12-17 13:04:59,838 [MainThread  ] [INFO ]  {"fit_val_acc": 0.936114733009844, "fit_test_dt_duration": "3", "fit_test_acc": 0.8193979933110368, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "219", "fit_val_loss": 0.21787163125644918, "fit_train_dt_end": "2018-12-17 13:04:56", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 13:04:56", "model_dropout": 0.2, "model_layer_1_sizefactor": 8, "fit_train_dt_start": "2018-12-17 13:01:16", "fit_batch_size": 32, "fit_train_acc": 0.9071267663321728, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 2,

2018-12-17 13:19:47,143 [MainThread  ] [INFO ]  begin experiment 377/512 - model_id: 377, architecture: LSTM, layer_1_sizefactor: 8, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 3, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 13:22:03,831 [MainThread  ] [INFO ]  model 377 test acc: 0.8662207357859532
2018-12-17 13:22:03,833 [MainThread  ] [INFO ]  {"fit_val_acc": 0.9191655804674711, "fit_test_dt_duration": "4", "fit_test_acc": 0.8662207357859532, "fit_num_classes": 2, "shape_x_valid": "(2301, 3, 1536)", "fit_stopper_patience": 10, "fit_train_dt_duration": "131", "fit_val_loss": 0.26363917402939296, "fit_train_dt_end": "2018-12-17 13:21:59", "shape_y_train": "(9766, 2)", "fit_test_dt_start": "2018-12-17 13:21:59", "model_dropout": 0.2, "model_layer_1_sizefactor": 8, "fit_train_dt_start": "2018-12-17 13:19:47", "fit_batch_size": 32, "fit_train_acc": 0.8855211959738676, "shape_x_test": "(299, 3, 1536)", "model_layer_3_sizefactor": 0

# Examine results

In [33]:
results = []
for folder, subs, files in os.walk(path_models):        
    for filename in files:
        if filename == 'params.json':
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)

results.sort_values("fit_test_acc", inplace=True, ascending=False)

In [39]:
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [40]:
results.head(20).T

,131,283,41,100,271,320,301,307,264,166,143,299,13,227,142,104,65,25,226,171
dt_duration_seconds,155,829,734,794,423,278,126,121,96,542,702,152,123,286,108,119,96,127,103,554
dt_end,2018-12-16 19:15:04,2018-12-16 22:20:45,2018-12-16 23:24:46,2018-12-16 21:03:18,2018-12-17 09:56:39,2018-12-17 09:24:11,2018-12-16 17:37:25,2018-12-16 17:22:01,2018-12-16 17:25:50,2018-12-16 22:37:01,2018-12-16 20:21:01,2018-12-17 10:45:58,2018-12-16 18:22:20,2018-12-17 00:30:27,2018-12-16 16:43:26,2018-12-16 18:25:56,2018-12-16 16:53:40,2018-12-16 18:36:05,2018-12-16 17:01:21,2018-12-17 00:14:33
dt_start,2018-12-16 19:12:29,2018-12-16 22:06:55,2018-12-16 23:12:32,2018-12-16 20:50:04,2018-12-17 09:49:36,2018-12-17 09:19:33,2018-12-16 17:35:18,2018-12-16 17:19:59,2018-12-16 17:24:14,2018-12-16 22:27:59,2018-12-16 20:09:18,2018-12-17 10:43:26,2018-12-16 18:20:17,2018-12-17 00:25:40,2018-12-16 16:41:38,2018-12-16 18:23:57,2018-12-16 16:52:04,2018-12-16 18:33:57,2018-12-16 16:59:37,2018-12-17 00:05:18
fit_batch_size,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
fit_num_classes,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
fit_num_epochs,14,35,28,35,36,20,16,13,10,18,30,16,12,16,15,10,12,12,12,17
fit_stopped_early,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
fit_stopper_patience,5,10,10,10,10,10,5,5,5,10,10,10,5,10,5,5,5,5,5,10
fit_test_acc,0.82274,0.92308,0.86288,0.87291,0.88294,0.87625,0.92977,0.92977,0.9097,0.81271,0.91973,0.86957,0.84615,0.8194,0.94314,0.71906,0.94649,0.75585,0.90635,0.8495
fit_train_acc,0.92075,0.92054,0.9103,0.91808,0.9146,0.90877,0.92965,0.91378,0.91327,0.89832,0.92228,0.90375,0.91634,0.89914,0.91644,0.90528,0.91839,0.92402,0.90723,0.90334


In [ ]:
results.to_csv(pwd+'results.csv')

# Analyze results of balanced vs unbalanced fits

In [ ]:
# results2 = pd.read_csv(pwd + 'results/results1.csv', index_col=0)

# results['type'] = 'unbalanced'
# results2['type'] = 'balanced'

# results = pd.concat([results,results2],axis=0)

# results.head().T